In [1]:
from decode import *

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

#### It seems that, with AlphaMonarch model, the phenomenom is not obvious on the displayed cases.

In [3]:
query="A coin is heads up. Fletcher flips the coin. Conception flips the coin. Is the coin still heads up?"
template = """[INST]{question}[/INST]
               """

# k_response = get_k_path_prob(model, tokenizer, template.format(question=query), k=1)
k_response = get_k_path_prob_follow_up(model, tokenizer, template.format(question=query), k=5)

[('5', tensor(0.0957)), ('heads,', tensor(0.2377)), ('4', tensor(0.1666)), ('tails.', tensor(3.5910e-05)), ('', tensor(0.0006)), ('', tensor(0.0008)), ('"', tensor(0.0829)), ('"No', tensor(2.3499e-05)), ('"No,"', tensor(0.0001)), ('says', tensor(8.2030e-06)), ('Con', tensor(9.3868e-06)), ('Conception', tensor(2.2744e-05)), ('Conception.', tensor(0.0002)), ('', tensor(0.0002)), ('', tensor(0.0001)), ('"', tensor(0.0118)), ('"Yes', tensor(9.9870e-05)), ('"Yes,"', tensor(0.0007)), ('says', tensor(0.0003)), ('F', tensor(0.0005)), ('Flet', tensor(0.0006)), ('Fletcher', tensor(0.0016)), ('Fletcher.', tensor(0.0006)), ('', tensor(0.0020)), ('', tensor(0.0070)), ('"', tensor(0.6216)), ('"No', tensor(0.0002)), ('"No,"', tensor(0.0011)), ('says', tensor(0.0004)), ('Con', tensor(0.0005)), ('Conception', tensor(0.0007)), ('Conception.', tensor(0.0018)), ('', tensor(0.0007)), ('', tensor(0.0021)), ('"', tensor(0.2540)), ('"Yes', tensor(0.0005))]
--------------------
[('', tensor(0.0949)), ('', tens

### Zephyr-7B: Yes / No / Unsure about whether A is better than B in attribute

In [2]:
# attributes = ['Rude_Customer']
# gt_attribute_to_pred_attribute = {"Which customer is more rude?": "Rude_Customer"}
compare_attributes = ["Do you think customer in conversation A is more rude?", "Do you think customer in conversation A is more polite?"]
conversation_pairs = ("Sales: Hello! Customer: Nice to meet you ;>",
                      "Sales: Hello! Would you like to know our insurance product? Customer: Not at all interested.")

query_template = """Compare customers' response in the two conversations: 
Conversation A: {conversation_a}
Conversation B: {conversation_b}
{compare_query}
Your answer: """

query = query_template.format(compare_query=compare_attributes[0], conversation_a=conversation_pairs[0], conversation_b=conversation_pairs[1])
possible_answers = ["Yes", "No", "Unsure"]
    
single_choice_response(model, tokenizer, query, possible_answers)   # 4.2s for one attribute, 42s for 10 attributes | OpenAI takes 30s for 10 attributes, but can get stuck

[tensor(0.0790), tensor(0.9185), tensor(0.0025)]

In [12]:
scores

[[tensor(0.8302), tensor(0.1660), tensor(0.0038)],
 [tensor(0.2285), tensor(0.7691), tensor(0.0024)]]

In [7]:
from typing import Tuple, List
def pairmatch_open(conversation_pairs: Tuple[str, str],
                   compare_attributes: List[str],
                   query_template = query_template,
                   possible_answers = ["Yes", "No", "Unsure"],
                   model = model,
                   tokenizer = tokenizer) -> List[dict]:
    """
    Open-sourced LLM based Multi-Attribute Comparative Rater
    """
    results = []
    for compare_attribute in compare_attributes:
        query = query_template.format(compare_query=compare_attribute, conversation_a=conversation_pairs[0], conversation_b=conversation_pairs[1])
        result = single_choice_response(model, tokenizer, query, possible_answers)
        results.append(result)
        
    return results

In [10]:
scores = pairmatch_open(conversation_pairs, compare_attributes)

[[tensor(0.8302), tensor(0.1660), tensor(0.0038)],
 [tensor(0.2285), tensor(0.7691), tensor(0.0024)]]

In [ ]:
# compare attribute with a Yes and No, that's it



template = """[INST] Here is a recording of a person's preference on a few items. 
{preferences}
Do you think this person would prefer {item1} over {item2}? Answers with yes, no, or unsure.[/INST]
           """
train_indices = [0,1,2,3,4]
test_index = 5
preferences = form_preference(Pset, train_indices)
query = template.format(preferences=preferences, item1=Pset[test_index][0], item2=Pset[test_index][1])

prob_yes = check_response_prob(model, tokenizer, query, target_response="yes")
prob_no = check_response_prob(model, tokenizer, query, target_response="no")

# normalized answer
norm_yes, norm_no = prob_yes / (prob_yes + prob_no), prob_no / (prob_yes + prob_no)
norm_yes, norm_no

In [1]:
# load model and tokenizer
from filter import *

tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-Mistral')
model = AutoModel.from_pretrained('Salesforce/SFR-Embedding-Mistral')
tokenizer.add_eos_token = True

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
def generate_synthetic_conversation(scenarios, customer_profiles, agent_profiles, model, num_samples):
    pass
customer_profiles = []
agent_profiles = []
num_samples = 0

### Scenario-Based Generation

In [27]:
# Scenario Based Generation
scenarios = [
    {
        "name": "RudeCustomer",
        "description": "Sales agent discuss insurance policy with a customer",
        "good_response": "Customer is rude.",
        "bad_response": "Customer is polite."
    },
]

conversations = generate_synthetic_conversation(scenarios, customer_profiles, agent_profiles, model, num_samples)

### Requirement: -- Customer should be Rude

In [2]:
max_length = 4096

pset = ["Sale: Hello, how can I help with your insurance needs today? Customer: Not interested",
 "Sale: Hello, how can I help with your insurance needs today? Customer: I am interested, but I am not sure if I can afford it",
 "Sale: Hello, how can I help with your insurance needs today? Customer: What have you got?",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Nah, I'll pass on that.",
 "Sale: Hello, how can I help with your insurance needs today? Customer: I've been thinking about getting insurance, what plans do you offer?",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk.",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk currently.",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk now.",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk right now.",]

batch_dict = tokenizer(pset, max_length=max_length-1, padding=True, truncation=True, return_tensors="pt")
outputs = model(**batch_dict)
embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1)
embeddings = embeddings.detach().numpy()

In [3]:
filter = ClusterFilter(0.7)
filtered_indices, filtered_embeddings = filter.pick_k_per_cluster(embeddings)
img = present_filtering_result(embeddings, filtered_embeddings)
filtered_pset = [pset[i] for i in filtered_indices]

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, af

In [21]:
question = "Sales: Hello, how can I help with your insurance needs today?"
responses = [conversation.split("Customer: ")[1] for conversation in filtered_pset]
import json
json.dumps(responses)
with open('responses.json', 'w') as json_file:
    json.dump(responses, json_file)


In [25]:
filtered_pset
file = []
for i in range(len(filtered_pset)):
    file.append({"conversation": filtered_pset[i], "label":["negative"]})
with open('filtered_pset.json', 'w') as json_file:
    json.dump(file, json_file)

In [9]:
# Now, time for the decoding time multi-attribute comparison test

# So you've got a bunch of diverse conversations, so what you idiot!!!??
# Now you get to annotate on these conversations !! -- with some annotator tool
# How am I supposed to annotate these Long conversations to you moron!!!!?????
# You use fucking response segment & conversation compression you idiot!! Just like how human does it!! You fold stuff!!!!!
# Fucking idiot !!!














In [ ]:
"Sale: Hello, how can I help with your insurance needs today? Customer: Not interested" 
"Sale: Hello, how can I help with your insurance needs today? Customer: I am interested, but I am not sure if I can afford it"
"Sale: Hello, how can I help with your insurance needs today? Customer: What have you got?"
"Sale: Hello, how can I help with your insurance needs today? Customer: Nah, I'll pass on that."
"Sale: Hello, how can I help with your insurance needs today? Customer: I've been thinking about getting insurance, what plans do you offer?"
"Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk."

In [29]:

filtered_pset

# We want to display the judgement result from the auto-evaluator
# We want to collect human feedback on the evaluation result
# We want to optimise our evaluator, based on the feedback


['Sale: Hello, how can I help with your insurance needs today? Customer: Not interested',
 'Sale: Hello, how can I help with your insurance needs today? Customer: I am interested, but I am not sure if I can afford it',
 'Sale: Hello, how can I help with your insurance needs today? Customer: What have you got?',
 "Sale: Hello, how can I help with your insurance needs today? Customer: Nah, I'll pass on that.",
 "Sale: Hello, how can I help with your insurance needs today? Customer: I've been thinking about getting insurance, what plans do you offer?",
 "Sale: Hello, how can I help with your insurance needs today? Customer: Sorry, I'm busy right now. Can't talk."]

In [2]:
# Turn that open-sourced LLM into a reward model (!)

# Toy exmample of human preference dataset: (Prefer red stuff over blue stuff, this preference shall be hidden, and never explicitly stated)
# preference dataset
Pset = [("Red Apple", "Blue Apple"), 
        ("Lava", "Sea"),
        ("Blood", "Sky"),
        ("Strawberry", "Blueberry"),
        ("Red Dragon", "Blue Dragon"),
        ("Red Wine", "Ocean"),
        ("Manchester United", "Chelsea")]


format_preference_pair = lambda x: f"{x[0]} > {x[1]}"
form_preference = lambda Pset, indices = range(len(Pset)): ('\n').join(['-'+format_preference_pair(Pset[i]) for i in indices])
preferences = form_preference(Pset, [0, 1, 2, 3, 4])


1. Directly Feed the preference information to the whatever Monarch 7B model (which verylikely overfit on the validation set), and the model does NOT get the basic preference through demonstration. This will at least make the prompt optimisation difficult. 

2. The problem is always on whether we can train the RM, and not about whether it solves itself from the start

In [7]:
# model
# In the deepmind paper, the embedder is token out and re-train on a classifier, I will try to do decoding trick with it


# Model looks at the preference dataset and tries to learn the preference, and then use that to generate a reward model

template = """[INST] Here is a recording of a person's preference on a few items. 
{preferences}
Do you think this person would prefer {item1} over {item2}? Answers with yes, no, or unsure.[/INST]
           """
train_indices = [0,1,2,3,4]
test_index = 5
preferences = form_preference(Pset, train_indices)
query = template.format(preferences=preferences, item1=Pset[test_index][0], item2=Pset[test_index][1])

prob_yes = check_response_prob(model, tokenizer, query, target_response="yes")
prob_no = check_response_prob(model, tokenizer, query, target_response="no")

# normalized answer
norm_yes, norm_no = prob_yes / (prob_yes + prob_no), prob_no / (prob_yes + prob_no)
norm_yes, norm_no


(tensor(0.8487), tensor(0.1513))

In [3]:
import tree_of_thoughts
from tree_of_thoughts import T

ImportError: cannot import name 'TreeOfThoughts' from 'tree_of_thoughts' (/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/tree_of_thoughts/__init__.py)

In [1]:
# ToT 
import os
from tree_of_thoughts import ToTAgent, MonteCarloSearch
from dotenv import load_dotenv
from swarms import Agnet, OpenAIChat

load_dotenv()


ImportError: cannot import name 'ToTAgent' from 'tree_of_thoughts' (/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/tree_of_thoughts/__init__.py)

In [4]:
inputs = tokenizer([query], return_tensors="pt")
gen_out = model.generate(**inputs, output_scores=True, return_dict_in_generate=True, max_new_tokens=20, pad_token_id=tokenizer.eos_token_id)
tokenizer.batch_decode(gen_out['sequences'], skip_special_tokens=True)

["[INST] Here is a recording of a person's preference on a few items. \n-Red Apple > Blue Apple\n-Lava > Sea\n-Blood > Sky\n-Strawberry > Blueberry\n-Red Dragon > Blue Dragon\nDo you think this person would prefer Red Wine over Ocean? Answers with yes, no, or unsure.[/INST]\n           \n[ASST] Based on the given preferences, it is unclear whether the person would prefer"]

In [1]:
# Then these are the preference score of the pair of items
norm_yes, norm_no
# So, the most naive way is to just use the current model to generate a preference score, and then fit that to the annotator with a linear matrix
# or, we would directly fine-tune the model on the preference dataset (which is not a good idea, because the model is very likely to overfit on the preference dataset)
# ok, but we can augment the preference dataset through prompt optimisation, and then fine-tune the model on the augmented dataset (such dataset could also be filtered to enhance diversity)




NameError: name 'norm_yes' is not defined